In [1]:
from load_circuits import QuantumCircuit
from artensor import (
    AbstractTensorNetwork, 
    ContractionTree, 
    find_order, 
    contraction_scheme,
    tensor_contraction,
    contraction_scheme_sparse,
    tensor_contraction_sparse
)
from copy import deepcopy
import numpy as np
import torch
torch.backends.cuda.matmul.allow_tf32 = False

Firstly, we load the sycamore quantum circuit with $n=30$, $m=14$ and EFGH sequence. `sc_target=30` means the largest size of intermediate tensors is $2^{30}$ (since the data type is `complex64`, it will take about 8G of memory). Thus, in order to perform such a contraction, you need a GPU with memory larger than 24G (`einsum` operator in pytorch need to take 3 times of involving tensors).

In [2]:
n, m, seq, device, sc_target, seed = 30, 14, 'EFGH', 'cuda', 30, 0
qc = QuantumCircuit(n, m, seq=seq)
edges = []
for i in range(len(qc.neighbors)):
    for j in qc.neighbors[i]:
        if i < j:
            edges.append((i, j))
neighbors = list(qc.neighbors)
final_qubits = set(range(len(neighbors) - n, len(neighbors)))
tensor_bonds = {
    i: [edges.index((min(i, j), max(i, j))) for j in neighbors[i]] 
    for i in range(len(neighbors)) if i not in final_qubits
} # open tensor network without final state
bond_dims = {i:2.0 for i in range(len(edges))}

Contract the open tensor network corresponding to the quantum circuit, to get the overall $2^{30}$ amplitudes.

In [3]:
order_slicing, slicing_bonds, ctree_new = find_order(
    tensor_bonds, bond_dims, seed, 
    sc_target=sc_target, trials=5, 
    iters=10, slicing_repeat=1, betas=np.linspace(3.0, 21.0, 61)
)
print('order_slicing =', order_slicing)
print('slicing_bonds =', slicing_bonds)

tensors = []
for x in range(len(qc.tensors)):
    if x not in final_qubits:
        tensors.append(qc.tensors[x].to(device))

scheme, bonds_final = contraction_scheme(ctree_new)

final_qubits = sorted(final_qubits)
permute_dims = [0] * len(final_qubits)
for x in range(len(bonds_final)):
    _, y = edges[bonds_final[x]]
    permute_dims[list(final_qubits).index(y)] = x
full_amps = tensor_contraction(
    deepcopy(tensors), scheme
).permute(permute_dims).reshape(-1).cpu()

order_slicing = [(0, 22), (1, 11), (24, 34), (0, 47), (2, 10), (3, 13), (1, 12), (23, 36), (0, 24), (2, 35), (1, 3), (0, 23), (60, 71), (59, 70), (50, 58), (2, 48), (0, 1), (60, 81), (59, 83), (50, 69), (28, 38), (0, 2), (59, 60), (17, 27), (41, 76), (28, 50), (0, 59), (5, 18), (17, 40), (26, 37), (29, 41), (0, 28), (5, 7), (17, 26), (0, 29), (5, 54), (0, 17), (52, 65), (64, 74), (9, 19), (0, 5), (52, 64), (9, 20), (8, 21), (4, 14), (6, 15), (0, 52), (75, 85), (32, 45), (8, 9), (46, 56), (4, 6), (0, 75), (32, 33), (55, 68), (31, 44), (30, 42), (8, 46), (25, 49), (4, 16), (0, 107), (57, 82), (43, 53), (32, 80), (55, 67), (30, 31), (8, 66), (4, 25), (0, 57), (88, 94), (61, 77), (32, 43), (55, 93), (8, 30), (62, 78), (0, 4), (39, 88), (32, 61), (8, 55), (51, 62), (0, 39), (8, 32), (73, 84), (51, 72), (0, 8), (96, 104), (63, 73), (0, 51), (63, 96), (95, 105), (79, 92), (0, 63), (95, 116), (97, 106), (0, 79), (95, 97), (117, 128), (118, 130), (86, 113), (91, 102), (0, 95), (117, 118), (86, 

Demonstration of the relation between slicing edges and fidelity.

In [7]:
slicing_edges_manually_select = [
    (44, 66), (46, 66), (69, 94), (81, 94), 
    (88, 99), (94, 116), (111, 127), (112, 122)
]

tensors_slicing = deepcopy(tensors)
slicing_indices = {}
neighbors_copy = deepcopy(neighbors)
tensor_network = AbstractTensorNetwork(
    deepcopy(tensor_bonds), 
    deepcopy(bond_dims))

while len(slicing_edges_manually_select):
    slicing_edge = slicing_edges_manually_select.pop(0)
    x, y = slicing_edge
    idx_x_y = neighbors_copy[x].index(y)
    idx_y_x = neighbors_copy[y].index(x)
    neighbors_copy[x].pop(idx_x_y)
    neighbors_copy[y].pop(idx_y_x)
    slicing_indices[(x, y)] = (idx_x_y, idx_y_x)
    tensors_slicing[x] = tensors_slicing[x].select(idx_x_y, 0)
    tensors_slicing[y] = tensors_slicing[y].select(idx_y_x, 0)

    tensor_network.slicing(edges.index(slicing_edge))
    ctree_appro = ContractionTree(deepcopy(tensor_network), order_slicing, 0)
    scheme, _ = contraction_scheme(ctree_appro)
    appro_amps = tensor_contraction(
        deepcopy(tensors_slicing), scheme
    ).permute(permute_dims).reshape(-1).cpu()
    fidelity = (
        (full_amps.conj() @ appro_amps.reshape(-1)).abs() /
        (full_amps.abs().square().sum().sqrt() * appro_amps.abs().square().sum().sqrt())
    ).square().item()
    
    print(
        'after slicing {} edges, fidelity now is {:.5f} (estimated value {})'.format(
            len(slicing_indices), fidelity, 1/2**(len(slicing_indices))
        )
    )


after slicing 1 edges, fidelity now is 0.49384 (estimated value 0.5)
after slicing 2 edges, fidelity now is 0.26454 (estimated value 0.25)
after slicing 3 edges, fidelity now is 0.13284 (estimated value 0.125)
after slicing 4 edges, fidelity now is 0.06635 (estimated value 0.0625)
after slicing 5 edges, fidelity now is 0.03350 (estimated value 0.03125)
after slicing 6 edges, fidelity now is 0.01619 (estimated value 0.015625)
after slicing 7 edges, fidelity now is 0.00608 (estimated value 0.0078125)
after slicing 8 edges, fidelity now is 0.00304 (estimated value 0.00390625)


Load the amplitudes calculated by Google using Schrodinger-Feynman algorithm as the ground truth.

In [8]:
def read_samples(filename):
    import os
    if os.path.exists(filename):
        samples_data = []
        with open(filename, 'r') as f:
            l = f.readlines()
        f.close()
        for line in l:
            ll = line.split()
            samples_data.append((ll[0], float(ll[1]) + 1j*float(ll[2])))
        return samples_data
    else:
        raise ValueError("{} does not exist".format(filename))

data = read_samples('amplitudes_n30_m14_s0_e0_pEFGH_10000.txt')
max_bitstrings = 1_000
bitstrings = [data[i][0] for i in range(max_bitstrings)]
amplitude_google = np.array([data[i][1] for i in range(max_bitstrings)])

Show that the result calculated by sparse-state is identical to Goggle's result.

In [9]:
tensors_sparsestate = []
for i in range(len(qc.tensors)):
    if i in final_qubits:
        tensors_sparsestate.append(
            torch.tensor([[1, 0], [0, 1]], dtype=torch.complex64, device=device)
        )
    else:
        tensors_sparsestate.append(qc.tensors[i].to(device))

tensor_bonds = {
    i:[edges.index((min(i, j), max(i, j))) for j in neighbors[i]] 
    for i in range(len(neighbors))
} # now all tensors will be included

order_slicing, slicing_bonds, ctree_new = find_order(
    tensor_bonds, bond_dims, final_qubits, seed, max_bitstrings, 
    sc_target=sc_target, trials=5, iters=10, slicing_repeat=1, 
    betas=np.linspace(3.0, 21.0, 61)
)

scheme_sparsestate, _, bitstrings_sorted = contraction_scheme_sparse(
    ctree_new, bitstrings, sc_target=sc_target
)

slicing_edges = [edges[i] for i in slicing_bonds]
slicing_indices = {}.fromkeys(slicing_edges)
neighbors_copy = deepcopy(neighbors)
for i, j in slicing_edges:
    idxi_j = neighbors_copy[i].index(j)
    idxj_i = neighbors_copy[j].index(i)
    neighbors_copy[i].pop(idxi_j)
    neighbors_copy[j].pop(idxj_i)
    slicing_indices[(i, j)] = (idxi_j, idxj_i)


amplitude_sparsestate = torch.zeros(
    (len(bitstrings),), dtype=torch.complex64, device=device
)
for s in range(2**len(slicing_edges)):
    configs = list(map(int, np.binary_repr(s, len(slicing_edges))))
    sliced_tensors = tensors_sparsestate.copy()
    for i in range(len(slicing_edges)):
        m, n = slicing_edges[i]
        idxm_n, idxn_m = slicing_indices[(m, n)]
        sliced_tensors[m] = sliced_tensors[m].select(idxm_n, configs[i]).clone()
        sliced_tensors[n] = sliced_tensors[n].select(idxn_m, configs[i]).clone()
    amplitude_sparsestate += tensor_contraction_sparse(
        sliced_tensors, scheme_sparsestate
    )

correct_num = 0
for i in range(len(bitstrings_sorted)):
    ind_google = bitstrings.index(bitstrings_sorted[i])
    relative_error = abs(
        amplitude_sparsestate[i].item() - amplitude_google[ind_google]
    ) / abs(amplitude_google[ind_google])
    if relative_error <= 0.05:
        correct_num += 1
print(f'bitstring amplitude correct ratio:{correct_num/max_bitstrings}')


bitstring amplitude correct ratio:0.995
